In [7]:
import pandas as pd
import numpy as np
train = pd.read_csv("criminal_train.csv")
train= train.drop(train.index[[44281,19230]])
test = pd.read_csv("criminal_test.csv")
train_y = train.Criminal
train_x = train.drop(['Criminal','PERID'],axis=1)
test_x= test.drop('PERID',axis=1)
train.describe()

,PERID,IFATHER,NRCH17_2,IRHHSIZ2,IIHHSIZ2,IRKI17_2,IIKI17_2,IRHH65_2,IIHH65_2,PRXRETRY,...,TOOLONG,TROUBUND,PDEN10,COUTYP2,MAIIN102,AIIND102,ANALWT_C,VESTR,VEREP,Criminal
count,4.571600e+04,45716.000000,45716.000000,45716.000000,45716.000000,45716.000000,45716.000000,45716.000000,45716.000000,45716.000000,...,45716.000000,45716.000000,45716.000000,45716.000000,45716.000000,45716.000000,45716.000000,45716.000000,45716.000000,45716.000000
mean,5.445304e+07,3.355740,0.476551,3.426568,1.001794,2.084259,1.007525,1.162700,1.011112,97.399248,...,2.219551,2.235082,1.646251,1.764787,1.979066,1.978870,4692.866519,40025.490135,1.493963,0.069451
std,2.553876e+07,1.176324,0.888438,1.427151,0.059868,1.102823,0.122446,0.468821,0.145842,12.338273,...,5.295857,5.293723,0.618168,0.771211,0.143164,0.143821,5724.700526,14.626635,0.499969,0.254221
min,1.000222e+07,1.000000,-1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.621832,40001.000000,1.000000,0.000000
25%,3.232899e+07,4.000000,0.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,99.000000,...,2.000000,2.000000,1.000000,1.000000,2.000000,2.000000,1252.472995,40013.000000,1.000000,0.000000
50%,5.410588e+07,4.000000,0.000000,3.000000,1.000000,2.000000,1.000000,1.000000,1.000000,99.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2719.634952,40025.000000,1.000000,0.000000
75%,7.612399e+07,4.000000,1.000000,4.000000,1.000000,3.000000,1.000000,1.000000,1.000000,99.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,5765.981068,40039.000000,2.000000,0.000000
max,9.999956e+07,4.000000,3.000000,6.000000,3.000000,4.000000,3.000000,3.000000,3.000000,99.000000,...,98.000000,98.000000,3.000000,3.000000,2.000000,2.000000,109100.623000,40050.000000,2.000000,1.000000


In [2]:
from sklearn.model_selection import train_test_split
train_x,val_x,train_y,val_y = train_test_split(train_x,train_y,random_state = 0)

In [8]:
from sklearn.preprocessing import Imputer
my_imputer = Imputer(missing_values= -1,strategy= 'median')

In [9]:
imputed_train_x=my_imputer.fit_transform(train_x)
imputed_test_x=my_imputer.transform(test_x)
imputed_val_x=my_imputer.transform(val_x)


In [63]:

from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
for i in range(10):
    i=i+95
    
    my_model = XGBRegressor(max_depth=5,n_estimators=i)
    my_model.fit(imputed_train_x,train_y,verbose=False)
    prediction= my_model.predict(imputed_val_x)
    submit= np.round(prediction,0)
    mae = mean_absolute_error(val_y,submit)
    print(mae)

0.0507466168922
0.0507466168922
0.050979934671
0.050979934671
0.050979934671
0.050979934671
0.0505132991134
0.050396640224
0.050396640224
0.050396640224


In [15]:
from sklearn.ensemble import GradientBoostingClassifier
model=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.05, loss='deviance', max_depth=8,
              max_features=0.3, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=100, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)
model.fit(imputed_train_x,train_y, sample_weight=None, monitor=None)
prediction= model.predict(imputed_test_x)


In [74]:

from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
for i in range(10):
    i=i+5
    i=i*0.01
    for j in range(10):
        j=j+95
        for k in range(3):
            k=k+3
            my_model = XGBRegressor(max_depth=k,learning_rate=i, n_estimators=j)
            my_model.fit(imputed_train_x,train_y,verbose=False)
            prediction= my_model.predict(imputed_val_x)
            submit= np.round(prediction,0)
            mae = mean_absolute_error(val_y,submit)
            print(i, j, k, mae)

(0.05, 95, 3, 0.051096593560429303)
(0.05, 95, 4, 0.050863275781614557)
(0.05, 95, 5, 0.050629958002799812)
(0.05, 96, 3, 0.051096593560429303)
(0.05, 96, 4, 0.050863275781614557)
(0.05, 96, 5, 0.050629958002799812)
(0.05, 97, 3, 0.051096593560429303)
(0.05, 97, 4, 0.050863275781614557)
(0.05, 97, 5, 0.050629958002799812)
(0.05, 98, 3, 0.051096593560429303)
(0.05, 98, 4, 0.050863275781614557)
(0.05, 98, 5, 0.050629958002799812)
(0.05, 99, 3, 0.051096593560429303)
(0.05, 99, 4, 0.050863275781614557)
(0.05, 99, 5, 0.050746616892207184)
(0.05, 100, 3, 0.05097993467102193)
(0.05, 100, 4, 0.050863275781614557)
(0.05, 100, 5, 0.050629958002799812)
(0.05, 101, 3, 0.051096593560429303)
(0.05, 101, 4, 0.05097993467102193)
(0.05, 101, 5, 0.050513299113392439)
(0.05, 102, 3, 0.051096593560429303)
(0.05, 102, 4, 0.05097993467102193)
(0.05, 102, 5, 0.050513299113392439)
(0.05, 103, 3, 0.05097993467102193)
(0.05, 103, 4, 0.05097993467102193)
(0.05, 103, 5, 0.050629958002799812)
(0.05, 104, 3, 0.0509

(0.12, 100, 5, 0.050163322445170321)
(0.12, 101, 3, 0.051096593560429303)
(0.12, 101, 4, 0.051096593560429303)
(0.12, 101, 5, 0.050163322445170321)
(0.12, 102, 3, 0.051096593560429303)
(0.12, 102, 4, 0.051096593560429303)
(0.12, 102, 5, 0.050279981334577693)
(0.12, 103, 3, 0.051096593560429303)
(0.12, 103, 4, 0.051096593560429303)
(0.12, 103, 5, 0.050279981334577693)
(0.12, 104, 3, 0.051096593560429303)
(0.12, 104, 4, 0.051096593560429303)
(0.12, 104, 5, 0.050279981334577693)
(0.13, 95, 3, 0.050863275781614557)
(0.13, 95, 4, 0.050279981334577693)
(0.13, 95, 5, 0.050163322445170321)
(0.13, 96, 3, 0.05097993467102193)
(0.13, 96, 4, 0.050279981334577693)
(0.13, 96, 5, 0.050279981334577693)
(0.13, 97, 3, 0.050746616892207184)
(0.13, 97, 4, 0.050279981334577693)
(0.13, 97, 5, 0.050163322445170321)
(0.13, 98, 3, 0.050746616892207184)
(0.13, 98, 4, 0.050279981334577693)
(0.13, 98, 5, 0.050163322445170321)
(0.13, 99, 3, 0.050746616892207184)
(0.13, 99, 4, 0.050279981334577693)
(0.13, 99, 5, 0.

In [86]:
from xgboost import XGBClassifier
model=XGBClassifier(max_depth=5,learning_rate=0.09,n_estimators=97)

In [87]:
model.fit(imputed_train_x,train_y,verbose=False)
prediction= model.predict(imputed_val_x)

In [88]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(val_y,submit)
mae

0.048298188817919328

In [ ]:
RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
            max_depth=None, max_features=10, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=10, min_samples_split=3,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [92]:
from xgboost import XGBRegressor
my_model = XGBRegressor(max_depth=9,learning_rate=0.09,n_estimators=97,gamma=0.00009)
my_model.fit(imputed_train_x,train_y,verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=9e-05, learning_rate=0.09,
       max_delta_step=0, max_depth=9, min_child_weight=1, missing=None,
       n_estimators=97, n_jobs=1, nthread=None, objective='reg:linear',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [93]:
prediction= my_model.predict(imputed_val_x)
submit= np.round(prediction,0)

In [94]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(val_y,submit)
mae

0.048210692099046284

In [205]:
for i in range(5):
    print(2)

2
2
2
2
2


In [28]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(val_y,submit)
mae

0.05097993467102193

In [29]:
prediction= my_model.predict(imputed_test_x)
submit= np.round(prediction,0)

In [19]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(val_y,submit)
mae

0.051213252449836676

In [13]:
submission1 = pd.DataFrame({'Criminal': prediction,'PERID': test.PERID})
submission1.to_csv('gradientboosting.csv',index= False)